In [4]:
# note: we need to be authenticated via `gcloud auth login`
PROJECT = 'idc-lnq-000'

# CLI access

In [3]:
import os, time

In [3]:
t0 = time.time()
datasets = os.popen('gcloud beta healthcare datasets list --project chc-tcia').read().split()[3::3]
print(time.time() - t0, 'seconds')

2.699720859527588 seconds


In [4]:
t0 = time.time()
for d in datasets[0:1]:
    size = os.popen('gsutil -u '+PROJECT+' du -s gs://gcs-public-data--healthcare-tcia-'+d).read().split()[0]
    print(d, size)
# note: takes a long time
print(time.time() - t0, 'seconds')

4d-lung 183043903814
90.05417990684509 seconds


# Python access

In [1]:
import time
from google.cloud import storage
import google.oauth2.credentials

In [32]:
token = os.popen('gcloud auth print-access-token').read().strip('\n') # TODO: better way?
credentials = google.oauth2.credentials.Credentials(token)

In [33]:
c = storage.Client(project='chc-tcia', credentials=credentials)

In [34]:
t0 = time.time()
datasets = c.list_buckets()
print(time.time() - t0, 'seconds')

0.00016188621520996094 seconds


In [9]:
# getting dataset size
t0 = time.time()
for d in list(datasets)[0:1]:
    bucket = c.bucket(d.name, user_project=PROJECT)
    size = 0
    for b in bucket.list_blobs():
        size += b.size
    print(d, size)
print(time.time() - t0, 'seconds')

<Bucket: gcs-public-data--healthcare-tcia-4d-lung> 183043903814
44.924907207489014 seconds


## Now for the ct-lumph-nodes

In [35]:
search = 'ct-lymph-nodes'

t0 = time.time()
datasets = c.list_buckets()
for d in list(datasets):
    if search in d.name:
        selected_dataset = d.name
        break
print(time.time() - t0, 'seconds')

0.2256476879119873 seconds


In [36]:
bucket = c.bucket(selected_dataset, user_project=PROJECT)

In [12]:
# parsing the folder structure
t0 = time.time()
folders = []
for b in bucket.list_blobs():
    f_split = b.name.split('/')
    for i,f in enumerate(f_split):
        
        if len(folders) <= i:
            folders.append([])
            
        if not f in folders[i]:
            folders[i].append(f)
print(time.time() - t0, 'seconds')

127.93318319320679 seconds


In [22]:
studies = folders[1]
series = folders[2]
images = folders[3]

In [37]:
bucket.get_blob(b.name)

<Blob: gcs-public-data--healthcare-tcia-ct-lymph-nodes, dicom/61.7.99637024410815362534709947562712202035/61.7.161580241154366015366471722739845851315/61.7.99825910240087036989126920986284228754.dcm, 1541432897598168>

In [38]:
b.name

'dicom/61.7.99637024410815362534709947562712202035/61.7.161580241154366015366471722739845851315/61.7.99825910240087036989126920986284228754.dcm'

In [58]:
# all patients
series = 'dicom'
# patients = list(c.list_blobs(bucket, prefix=series, delimiter='.', max_results=10))
patients = list(c.list_blobs(bucket, delimiter='/', prefix='dicom/'))
print(len(patients))

0


In [59]:
patients = list(bucket.list_blobs(delimiter='/', prefix='dicom/'))


In [42]:
# for one series
series = 'dicom/61.7.99637024410815362534709947562712202035/61.7.161580241154366015366471722739845851315/'
dcm_files = list(c.list_blobs(bucket, prefix=series))
print(len(dcm_files))

481


In [60]:
patients

[]

In [77]:
def list_gcs_directories(client, bucket, prefix):
    # from https://github.com/GoogleCloudPlatform/google-cloud-python/issues/920
    iterator = client.list_blobs(bucket, prefix=prefix, delimiter='/')
    prefixes = set()
    for page in iterator.pages:
        print (page, page.prefixes)
        prefixes.update(page.prefixes)
    return prefixes

In [78]:
a = list_gcs_directories(c, bucket, 'dicom/')

<google.api_core.page_iterator.Page object at 0x7f161d86b6d0> ('dicom/61.7.102432584850582937613178565576727689438/', 'dicom/61.7.105477223282770249469045163366732744/', 'dicom/61.7.105967400867162609314453228373157076986/', 'dicom/61.7.106011913688873777967000368671613390694/', 'dicom/61.7.107323046883622997879712834941050736137/', 'dicom/61.7.107540433741703919518178117087316678228/', 'dicom/61.7.10809797054679529895122499097380370346/', 'dicom/61.7.109447583610681779128956367905974506260/', 'dicom/61.7.110976287009623783394893059131426578073/', 'dicom/61.7.112455531449190186311063143321694682940/', 'dicom/61.7.112606669699517023270746509553658743064/', 'dicom/61.7.116380062505676653966776255222512862129/', 'dicom/61.7.124675672310454047764468995145122534963/', 'dicom/61.7.126744359095913243515016888555680300804/', 'dicom/61.7.127749764394941233488146681623888696805/', 'dicom/61.7.128876040172251404065414822554792863476/', 'dicom/61.7.129139773030273642478494681211592832851/', 'dicom

In [79]:
len(a)

176

In [81]:
list(a)[0]

'dicom/61.7.27882698591755458810289506106138341571/'

In [82]:
a = list_gcs_directories(c, bucket, list(a)[0])

<google.api_core.page_iterator.Page object at 0x7f15fba08c40> ('dicom/61.7.27882698591755458810289506106138341571/61.7.140850485026854845569976770842695917955/',)


In [87]:
p = list(a)[0]

In [90]:
dcm_files = list(c.list_blobs(bucket, prefix=p))
print(len(dcm_files))

656


In [89]:
len(dicoms)

0

In [92]:
bb = dcm_files[0]

In [94]:
bb.download_to_file('/tmp/a.dcm')

RefreshError: The credentials do not contain the necessary fields need to refresh the access token. You must specify refresh_token, token_uri, client_id, and client_secret.

In [93]:
bb.name

'dicom/61.7.27882698591755458810289506106138341571/61.7.140850485026854845569976770842695917955/61.7.100009415753864105300316343655028182404.dcm'

In [104]:
credentials.refresh()


TypeError: refresh() missing 1 required positional argument: 'request'